In [ ]:
from main import profile_run, run_loop, push_next_collision
from utillities import check_particle_collision
%load_ext line_profiler

In [ ]:
%lprun -f profile_run profile_run()

In [ ]:
%lprun -f run_loop profile_run()

In [ ]:
%lprun -f push_next_collision profile_run()

In [ ]:
%lprun -f check_particle_collision profile_run()